Assignment: Flowers Recognition <br>
Dataset Description:<br>

This dataset contains 4242 images of flowers.<br>
The data collection is based on the data flicr, google images, yandex images.<br>
You can use this datastet to recognize plants from the photo.<br>

Attribute Information:<br>
The pictures are divided into five classes: chamomile, tulip, rose, sunflower, dandelion.<br>
For each class there are about 800 photos. Photos are not high resolution, about 320x240 pixels. <br>
<b>Also explore how to resize images in tensorflow and then resize all the images to a same size. </b> <br>
This is a Multiclass Classification Problem.<br>




WORKFLOW : <br>
Load Data <br>
Split into 60 and 40 ratio.<br>
Encode labels.<br>
Create Model<br>
Compilation Step (Note : Its a Multiclass Classification problem , select loss , metrics according to it)<br>
Train the Model.<br>
If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .<br>
Prediction should be > 85%<br>
Evaluation Step<br>
Prediction<br>




Data : <br>
https://drive.google.com/file/d/1-OX6wn5gA-bJpjPNfSyaYQLz-A-AB_uj/view?usp=sharing

## Connecting My Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Extracting Zip file directly Here

In [3]:
!unzip /content/drive/MyDrive/archive.zip

## Libarires

In [2]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, f1_score
from sklearn.preprocessing import LabelEncoder
%matplotlib inline
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile

#preprocess.
from keras.preprocessing.image import ImageDataGenerator

from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPool2D, Dropout,BatchNormalization
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from keras.optimizers import Adam


## Load Data

In [3]:
X=[]
Z=[]
IMG_SIZE=150

In [4]:
dais='/content/flowers/daisy'
sunflower='/content/flowers/sunflower'
tupli='/content/flowers/tulip'
dandi='/content/flowers/dandelion'
rose='/content/flowers/rose'

In [5]:
def flower_label(img,flower):
    return flower

In [6]:
def data(flower,DIR):
    for img in tqdm(os.listdir(DIR)):
      try:
        label=flower_label(img,flower)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        X.append(np.array(img))
        Z.append(str(label))
      except:
        pass

In [7]:
data('Daisy',dais)
print(len(X))

100%|██████████| 769/769 [00:02<00:00, 381.37it/s]

769


In [8]:
data('Sunflower',sunflower)
print(len(X))

100%|██████████| 734/734 [00:02<00:00, 304.54it/s]

1503


In [9]:
data('Tupli',tupli)
print(len(X))

100%|██████████| 984/984 [00:02<00:00, 356.73it/s]

2487


In [10]:
data('Dandi',dandi)
print(len(X))

100%|██████████| 1055/1055 [00:02<00:00, 364.54it/s]

3539


In [11]:
data('Rose',rose)
print(len(X))

100%|██████████| 784/784 [00:02<00:00, 388.65it/s]

4323


## Use Label Encoder for Label Data

In [12]:
le=LabelEncoder()
Y=le.fit_transform(Z)
Y=to_categorical(Y,5)

## Normalization

In [13]:
X=np.array(X)
X=X/255

## Split Dataset

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.40, random_state = 42)
print ("X_train: ", len(X_train))
print("X_test: ", len(X_test))
print("y_train: ", len(y_train))
print("y_test: ", len(y_test))

X_train:  2593
X_test:  1730
y_train:  2593
y_test:  1730


In [14]:
X_train.shape

(2593, 150, 150, 3)

## Model

In [18]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPool2D(2, 2, padding="same"))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPool2D(2, 2, padding="same"))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPool2D(2, 2, padding="same"))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(5, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 384)       885120    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 384)       1327488   
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 256)       8

## Complier

In [19]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/10
82/82 [==============================] - 135s 2s/step - loss: 1.7462 - accuracy: 0.2792 - val_loss: 1.2779 - val_accuracy: 0.4682
Epoch 2/10
82/82 [==============================] - 133s 2s/step - loss: 1.2395 - accuracy: 0.4750 - val_loss: 1.1321 - val_accuracy: 0.5254
Epoch 3/10
82/82 [==============================] - 133s 2s/step - loss: 1.1549 - accuracy: 0.5285 - val_loss: 1.0695 - val_accuracy: 0.5607
Epoch 4/10
82/82 [==============================] - 132s 2s/step - loss: 1.0446 - accuracy: 0.5888 - val_loss: 1.0158 - val_accuracy: 0.5936
Epoch 5/10
82/82 [==============================] - 132s 2s/step - loss: 0.9439 - accuracy: 0.6321 - val_loss: 1.0201 - val_accuracy: 0.5884
Epoch 6/10
82/82 [==============================] - 131s 2s/step - loss: 0.8107 - accuracy: 0.6781 - val_loss: 0.9866 - val_accuracy: 0.6133
Epoch 7/10
82/82 [==============================] - 132s 2s/step - loss: 0.6844 - accuracy: 0.7414 - val_loss: 0.9412 - val_accuracy: 0.6514
Epoch 8/10
82

## Prediction

In [21]:
pred = model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


## Evaluation


In [22]:
results = model.evaluate(X_test,y_test)
print('Test accuracy: ', results[1]*100)

55/55 [==============================] - 19s 343ms/step - loss: 1.4817 - accuracy: 0.6023
Test accuracy:  60.23121476173401
